In [20]:
!pip install PyPDF2 langchain faiss-cpu openai sentence-transformers

In [21]:
import os
from PyPDF2 import PdfReader
from langchain.embeddings import OpenAIEmbeddings, SentenceTransformerEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferWindowMemory
from langchain.chat_models import ChatOpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
import openai

# OpenAI API Key 설정
OPENAI_API_KEY = 'sk-'
openai.api_key = OPENAI_API_KEY
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [22]:
# 텍스트 파일 로드 함수
def load_text(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()

In [23]:
# PDF에서 텍스트 추출
def get_pdf_text(file_path):
    text = ""
    pdf_reader = PdfReader(file_path)
    for page in pdf_reader.pages:
        text += page.extract_text()
    return text


In [24]:
# 텍스트를 청크 단위로 분할 및 벡터 스토어 생성
def process_documents(documents, embeddings):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    text_chunks = text_splitter.split_text("\n".join(documents))
    vectorstore = FAISS.from_texts(text_chunks, embeddings)
    return vectorstore


In [25]:
# 대화 체인 생성
def get_conversation_chain(vectorstore):
    memory = ConversationBufferWindowMemory(memory_key='chat_history', return_messages=True, window_size=10)
    conversation_chain = ConversationalRetrievalChain.from_llm(
        llm=ChatOpenAI(temperature=0, model_name='gpt-3.5-turbo'),
        retriever=vectorstore.as_retriever(),
        memory=memory
    )
    return conversation_chain

In [26]:
# 파일 처리 함수
def process_files(file_paths):
    documents = []
    for file_path in file_paths:
        if file_path.endswith('.txt'):
            documents.append(load_text(file_path))
        elif file_path.endswith('.pdf'):
            documents.append(get_pdf_text(file_path))

    embeddings = SentenceTransformerEmbeddings(model_name='all-MiniLM-L6-v2')
    vectorstore = process_documents(documents, embeddings)
    return get_conversation_chain(vectorstore)

In [27]:
# 파일 경로 입력
file_paths = input("텍스트 또는 PDF 파일 경로를 쉼표로 구분하여 입력하세요: ").split(',')

In [28]:
# 파일 처리
conversation_chain = process_files(file_paths)
chat_history = []

In [29]:
# 대화 루프
print("질문을 입력하고 Enter 키를 누르세요. 종료하려면 'exit'를 입력하세요.")
while True:
    user_query = input("질문: ")
    if user_query.lower() == 'exit':
        break

    chat_history.append({"role": "user", "content": user_query})
    try:
        result = conversation_chain({
            "question": user_query,
            "chat_history": chat_history
        })
        response = result["answer"]
        chat_history.append({"role": "assistant", "content": response})
    except Exception as e:
        response = f"오류가 발생했습니다: {e}"
        chat_history.append({"role": "assistant", "content": response})

    print(f"답변: {response}")

질문을 입력하고 Enter 키를 누르세요. 종료하려면 'exit'를 입력하세요.
답변: 프롬프트를 잘 하는 방법은 다음과 같습니다:
1. 명확하고 구체적으로 작성하기
2. 맥락 제공하기
3. 간결하게 작성하기
4. 단계적으로 질문하기
5. 피드백 제공하기
6. 예시 포함하기
7. 열린 질문과 닫힌 질문 구분하기

이러한 방법을 활용하면 AI가 원하는 답변을 더 정확하게 제공할 수 있습니다.
답변: 프롬프트를 잘 하는 방법은 다음과 같습니다:
1. 명확하고 구체적으로 작성하기
2. 맥락 제공하기
3. 간결하게 작성하기
4. 단계적으로 질문하기
5. 피드백 제공하기
6. 예시 포함하기
7. 열린 질문과 닫힌 질문 구분하기
